## Архитектуры нейронок для рахных задач

In [ ]:
# 1. Координаты X, Y, TVD1 и TVD2 лежат отдельными листами в .xlsx файле
# 2. Файлов .tfrecord не пугаться, в деталях коммита на всякий указал как открывать такие. 
# Там 4 таких файла уже в порядке следования дат, для нефти, воды, пластового и забойного.
# Каждый тензор размерности [121,6,6]. 
# 3. Чтобы узнать какой дате соответствует индекс в тензоре, можно заюзать файл Dates_processed.csv.
# Загружаете как датафрейм и запрашиваете по индексу строки, получаете дату.

In [511]:
# imports 

import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LSTM, TimeDistributed, Input, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

### Обработка данных

In [321]:
# specify your directory where files are located 
# tensors loading

files_dir = '../data/processed/'
loaded_tensors = dict()
for file in [x for x in os.listdir(files_dir) if x.endswith("tfrecord")]:
    serialized_tensor = tf.io.read_file(os.path.join(files_dir,file))
    loaded_tensors[file] = tf.io.parse_tensor(serialized_tensor, out_type=tf.float32)

# coordinates loading 
df_coords = pd.read_excel(os.path.join(files_dir,"Coordinates_processed.xlsx"),sheet_name=['X','Y','TVD1','TVD2'],header=None)
df_X_Y = pd.DataFrame({col: list(zip(df_coords["X"][col], df_coords["Y"][col])) for col in  df_coords["X"].columns})
df_TVD = pd.DataFrame({col: list(zip(df_coords["TVD1"][col], df_coords["TVD2"][col])) for col in  df_coords["TVD1"].columns})

# time loading 
time_sequence = pd.read_csv(os.path.join(files_dir,"Dates_processed.csv"))

### Подготовка датасета для сплита 8/4:

In [ ]:
def prepare_data(N, W, P, Z, TVD1, TVD2, X, Y):
  input_data = []
  for i in range(len(W)):
    combined_input = tf.concat([tf.reshape(W[i], [-1]), tf.reshape(P[i], [-1]), tf.reshape(Z[i], [-1])], axis=0)
    input_data.append(combined_input)
  return tf.stack(input_data), N

input_data, target_data = prepare_data(N, W, P, Z, TVD1, TVD2, X, Y)

constant_input = tf.concat([tf.reshape(TVD1, [-1]), tf.reshape(TVD2, [-1]),
                            tf.reshape(X, [-1]), tf.reshape(Y, [-1])], axis=0)
constant_input = tf.expand_dims(constant_input, axis=0)

def generate_train_val_sets(input_data, target_data):
  train_inputs = []
  train_targets = []
  val_inputs = []
  val_targets = []

  i = 1
  while i + 12 <= len(input_data):
    train_inputs.append(input_data[i:i+8])
    train_targets.append(target_data[i+1:i+9])
    val_inputs.append(input_data[i+8:i+12])
    val_targets.append(target_data[i+9:i+13])
    i += 12

  return (tf.concat(train_inputs, axis=0), tf.concat(train_targets, axis=0),
          tf.concat(val_inputs, axis=0), tf.concat(val_targets, axis=0))

train_input, train_target, val_input, val_target = generate_train_val_sets(input_data, target_data)

train_input = tf.reshape(train_input, (train_input.shape[0], 1, -1))
val_input = tf.reshape(val_input, (val_input.shape[0], 1, -1))

### Увеличение данных

In [314]:
# import pandas as pd
# from pandas.tseries.offsets import MonthEnd

# # Creating a date range from December 1, 2020, to December 1, 2030 with monthly frequency
# monthly_range = pd.date_range(start='2020-12-01', end='2030-12-01', freq='MS')

# # Function to generate 5 evenly distributed days in each month
# def generate_evenly_distributed_days(monthly_dates):
#     extended_dates = []
#     for date in monthly_dates:
#         month_end = date + MonthEnd(1)
#         dates_in_month = pd.date_range(start=date, end=month_end, periods=5)
#         extended_dates.extend(dates_in_month)
#     return pd.to_datetime(extended_dates)

# # Applying the function to generate the desired dates
# evenly_distributed_dates = generate_evenly_distributed_days(monthly_range)

# # Creating a DataFrame to display
# evenly_distributed_date_df = pd.DataFrame(evenly_distributed_dates, columns=['Date'])

# # Remove the time part from the dates
# evenly_distributed_date_df['Date'] = evenly_distributed_date_df['Date'].dt.date


### Добавление приемистости 

In [533]:
raw_data_dir = '../data/'
df_injectivity = pd.read_excel(os.path.join(raw_data_dir,"raw_data.xlsx"),sheet_name=['приемистость'])
df_injectivity = df_injectivity["приемистость"].drop(columns=['Unnamed: 0'])

### Пример кода преобразования

In [ ]:
import numpy as np

# Initial matrix values
matrix_values = [
    ['p22', 'p23', 'p24', 'p30', 'p31', 'p32'],
    ['p20', 'INJE3', 'p21', 'p28', 'INJE4', 'p29'],
    ['p17', 'p18', 'p19', 'p25', 'p26', 'p27'],
    ['p6', 'p7', 'p8', 'p14', 'p15', 'p16'],
    ['p4', 'INJE1', 'p5', 'p12', 'INJE2', 'p13'],
    ['p1', 'p2', 'p3', 'p9', 'p10', 'p11']
]

# Convert to numpy array and repeat
matrix_array = np.array(matrix_values)
repeated_matrix = np.tile(matrix_array, (121, 1, 1))

# Create a mask to remove the specific elements
mask = np.ones(repeated_matrix.shape, dtype=bool)
mask[:, 1, 1] = False
mask[:, 1, 4] = False
mask[:, 4, 1] = False
mask[:, 4, 4] = False  # This seems to be a repeat; ensure this element is removed

# Apply the mask and reshape the array
filtered_array = repeated_matrix[mask].reshape(121, 4, 8)

# Display the shape to verify
print(filtered_array.shape)

In [ ]:
filtered_array[0]

### Преобразование массивов (121,6,6) в (121,4,8)

In [ ]:
oil_data = loaded_tensors['debit_neft_series.tfrecord'].numpy()
water_data = loaded_tensors['debit_water_series.tfrecord'].numpy()
reservoir_pressure_data = loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy()
bottomhole_pressure_data = loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy()

# Create a mask to remove the specific elements
mask = np.ones(oil_data.shape, dtype=bool)
mask[:, 1, 1] = False
mask[:, 1, 4] = False
mask[:, 4, 1] = False
mask[:, 4, 4] = False  # This seems to be a repeat; ensure this element is removed

# Apply the mask and reshape the array
oil_data = oil_data[mask].reshape(121, 4, 8)
water_data = water_data[mask].reshape(121, 4, 8)
reservoir_pressure_data = reservoir_pressure_data[mask].reshape(121, 4, 8)
bottomhole_pressure_data = bottomhole_pressure_data[mask].reshape(121, 4, 8)

# reshape coords and TVD
df_X_Y = df_X_Y.values[mask[0]].reshape(4, 8)
df_TVD = df_TVD.values[mask[0]].reshape(4, 8)

In [ ]:
filtered_array[0]

### Преобразование массивов (121,6,6) в (121,4,8)

In [ ]:
oil_data = loaded_tensors['debit_neft_series.tfrecord'].numpy()
water_data = loaded_tensors['debit_water_series.tfrecord'].numpy()
reservoir_pressure_data = loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy()
bottomhole_pressure_data = loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy()

# Create a mask to remove the specific elements
mask = np.ones(oil_data.shape, dtype=bool)
mask[:, 1, 1] = False
mask[:, 1, 4] = False
mask[:, 4, 1] = False
mask[:, 4, 4] = False  # This seems to be a repeat; ensure this element is removed

# Apply the mask and reshape the array
oil_data = oil_data[mask].reshape(121, 4, 8)
water_data = water_data[mask].reshape(121, 4, 8)
reservoir_pressure_data = reservoir_pressure_data[mask].reshape(121, 4, 8)
bottomhole_pressure_data = bottomhole_pressure_data[mask].reshape(121, 4, 8)

# reshape coords and TVD
df_X_Y = df_X_Y.values[mask[0]].reshape(4, 8)
df_TVD = df_TVD.values[mask[0]].reshape(4, 8)

### Преобразование массивов (121,6,6) в (121,4,8)

In [ ]:
oil_data = loaded_tensors['debit_neft_series.tfrecord'].numpy()
water_data = loaded_tensors['debit_water_series.tfrecord'].numpy()
reservoir_pressure_data = loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy()
bottomhole_pressure_data = loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy()

# Create a mask to remove the specific elements
mask = np.ones(oil_data.shape, dtype=bool)
mask[:, 1, 1] = False
mask[:, 1, 4] = False
mask[:, 4, 1] = False
mask[:, 4, 4] = False  # This seems to be a repeat; ensure this element is removed

# Apply the mask and reshape the array
oil_data = oil_data[mask].reshape(121, 4, 8)
water_data = water_data[mask].reshape(121, 4, 8)
reservoir_pressure_data = reservoir_pressure_data[mask].reshape(121, 4, 8)
bottomhole_pressure_data = bottomhole_pressure_data[mask].reshape(121, 4, 8)

# reshape coords and TVD
df_X_Y = df_X_Y.values[mask[0]].reshape(4, 8)
df_TVD = df_TVD.values[mask[0]].reshape(4, 8)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# Parameters
total_time_steps = 121
train_time_steps = 84
test_time_steps = total_time_steps - train_time_steps
map_height = 6
map_width = 6
batch_size = 1
dropout_rate = 0.3

# Function to create CNN layers for input tensors
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_tensor)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Input tensors for training model
inputs_train = {
    'oil': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(train_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Input tensors for testing model
inputs_test = {
    'oil': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(test_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Creating CNN layers for training and testing
cnn_train = {name: create_cnn(tensor) for name, tensor in inputs_train.items() if name not in ['coordinates', 'tvd', 'injectivity']}
cnn_test = {name: create_cnn(tensor) for name, tensor in inputs_test.items() if name not in ['coordinates', 'tvd', 'injectivity']}

# Flatten and repeat coordinates and TVD for training
flattened_coordinates_train = Flatten()(inputs_train['coordinates'])
repeated_coordinates_train = RepeatVector(train_time_steps)(flattened_coordinates_train)
flattened_tvd_train = Flatten()(inputs_train['tvd'])
repeated_tvd_train = RepeatVector(train_time_steps)(flattened_tvd_train)

# Flatten and repeat coordinates and TVD for testing
flattened_coordinates_test = Flatten()(inputs_test['coordinates'])
repeated_coordinates_test = RepeatVector(test_time_steps)(flattened_coordinates_test)
flattened_tvd_test = Flatten()(inputs_test['tvd'])
repeated_tvd_test = RepeatVector(test_time_steps)(flattened_tvd_test)

# Concatenating all inputs for training, including injectivity
merged_train = Concatenate(axis=-1)(list(cnn_train.values()))
flattened_merged_train = TimeDistributed(Flatten())(merged_train)
combined_train = Concatenate(axis=-1)([flattened_merged_train, repeated_coordinates_train, repeated_tvd_train, inputs_train['injectivity']])
# Example of adding more layers to the LSTM model
lstm_out_train = LSTM(128, return_sequences=True)(combined_train)
lstm_out_train = LSTM(64, return_sequences=False)(lstm_out_train)
dropout_train = Dropout(dropout_rate)(lstm_out_train)
dense_1_train = Dense(128, activation='relu')(dropout_train)
dropout_train_2 = Dropout(dropout_rate)(dense_1_train)
dense_2_train = Dense(72, activation='relu')(dropout_train_2)
output_train = Dense(map_height * map_width, activation='linear', name='output')(dense_2_train)
output_train = Reshape((map_height, map_width))(output_train)

# Training model
model_train = Model(inputs=list(inputs_train.values()), outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Concatenating all inputs for testing, including injectivity
merged_test = Concatenate(axis=-1)(list(cnn_test.values()))
flattened_merged_test = TimeDistributed(Flatten())(merged_test)
combined_test = Concatenate(axis=-1)([flattened_merged_test, repeated_coordinates_test, repeated_tvd_test, inputs_test['injectivity']])
lstm_out_test = LSTM(128, return_sequences=False)(combined_test)
dropout_test = Dropout(dropout_rate)(lstm_out_test)
dense_1_test = Dense(128, activation='relu')(dropout_test)
dropout_test_2 = Dropout(dropout_rate)(dense_1_test)
dense_2_test = Dense(72, activation='relu')(dropout_test_2)
output_test = Dense(map_height * map_width, activation='linear', name='output')(dense_2_test)
output_test = Reshape((map_height, map_width))(output_test)

# Testing model
model_test = Model(inputs=list(inputs_test.values()), outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Load and preprocess data (Assuming loaded_tensors and injectivity_array are already defined)
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Transforming df_X_Y and df_TVD to numpy arrays of required shape
coordinates_data = np.array(df_X_Y.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))
tvd_data = np.array(df_TVD.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))

# Ensuring data shapes match expected shape
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Проверка на наличие пропусков и заполнение нулями или средними значениями
oil_data = np.nan_to_num(oil_data)
water_data = np.nan_to_num(water_data)
reservoir_pressure_data = np.nan_to_num(reservoir_pressure_data)
bottomhole_pressure_data = np.nan_to_num(bottomhole_pressure_data)
injectivity_data = np.nan_to_num(injectivity_array.reshape((batch_size, total_time_steps, 4)))

# Масштабирование данных
scalers = {}
for data_name, data in zip(['oil', 'water', 'reservoir_pressure', 'bottomhole_pressure', 'injectivity'],
                           [oil_data, water_data, reservoir_pressure_data, bottomhole_pressure_data, injectivity_data]):
    scalers[data_name] = MinMaxScaler()
    data_reshaped = data.reshape(-1, data.shape[-1])
    scaled_data = scalers[data_name].fit_transform(data_reshaped)
    data[:] = scaled_data.reshape(data.shape)

# Splitting data into training and test sets
oil_train, oil_test = oil_data[:, :train_time_steps], oil_data[:, train_time_steps:]
water_train, water_test = water_data[:, :train_time_steps], water_data[:, train_time_steps:]
reservoir_pressure_train, reservoir_pressure_test = reservoir_pressure_data[:, :train_time_steps], reservoir_pressure_data[:, train_time_steps:]
bottomhole_pressure_train, bottomhole_pressure_test = bottomhole_pressure_data[:, :train_time_steps], bottomhole_pressure_data[:, train_time_steps:]

# Splitting injectivity data into training and test sets
injectivity_train, injectivity_test = injectivity_data[:, :train_time_steps], injectivity_data[:, train_time_steps:]

# Define output for training and testing
output_train_data = oil_data[:, :train_time_steps, :, :, 0].reshape((batch_size, train_time_steps, map_height, map_width))
output_test_data = oil_data[:, train_time_steps:, :, :, 0].reshape((batch_size, test_time_steps, map_height, map_width))

# Adding EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model with validation
history = model_train.fit(
    [oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train, coordinates_data, tvd_data, injectivity_train],
    output_train_data,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
oil_data = loaded_tensors['debit_neft_series.tfrecord'].numpy()
water_data = loaded_tensors['debit_water_series.tfrecord'].numpy()
reservoir_pressure_data = loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy()
bottomhole_pressure_data = loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy()

# Create a mask to remove the specific elements
mask = np.ones(oil_data.shape, dtype=bool)
mask[:, 1, 1] = False
mask[:, 1, 4] = False
mask[:, 4, 1] = False
mask[:, 4, 4] = False  # This seems to be a repeat; ensure this element is removed

# Apply the mask and reshape the array
oil_data = oil_data[mask].reshape(121, 4, 8)
water_data = water_data[mask].reshape(121, 4, 8)
reservoir_pressure_data = reservoir_pressure_data[mask].reshape(121, 4, 8)
bottomhole_pressure_data = bottomhole_pressure_data[mask].reshape(121, 4, 8)

# reshape coords and TVD
df_X_Y = df_X_Y.values[mask[0]].reshape(4, 8)
df_TVD = df_TVD.values[mask[0]].reshape(4, 8)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# Parameters
total_time_steps = 121
train_time_steps = 84
test_time_steps = total_time_steps - train_time_steps
map_height = 6
map_width = 6
batch_size = 1
dropout_rate = 0.3

# Function to create CNN layers for input tensors
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_tensor)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Input tensors for training model
inputs_train = {
    'oil': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(train_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Input tensors for testing model
inputs_test = {
    'oil': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(test_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Creating CNN layers for training and testing
cnn_train = {name: create_cnn(tensor) for name, tensor in inputs_train.items() if name not in ['coordinates', 'tvd', 'injectivity']}
cnn_test = {name: create_cnn(tensor) for name, tensor in inputs_test.items() if name not in ['coordinates', 'tvd', 'injectivity']}

# Flatten and repeat coordinates and TVD for training
flattened_coordinates_train = Flatten()(inputs_train['coordinates'])
repeated_coordinates_train = RepeatVector(train_time_steps)(flattened_coordinates_train)
flattened_tvd_train = Flatten()(inputs_train['tvd'])
repeated_tvd_train = RepeatVector(train_time_steps)(flattened_tvd_train)

# Flatten and repeat coordinates and TVD for testing
flattened_coordinates_test = Flatten()(inputs_test['coordinates'])
repeated_coordinates_test = RepeatVector(test_time_steps)(flattened_coordinates_test)
flattened_tvd_test = Flatten()(inputs_test['tvd'])
repeated_tvd_test = RepeatVector(test_time_steps)(flattened_tvd_test)

# Concatenating all inputs for training, including injectivity
merged_train = Concatenate(axis=-1)(list(cnn_train.values()))
flattened_merged_train = TimeDistributed(Flatten())(merged_train)
combined_train = Concatenate(axis=-1)([flattened_merged_train, repeated_coordinates_train, repeated_tvd_train, inputs_train['injectivity']])
# Example of adding more layers to the LSTM model
lstm_out_train = LSTM(128, return_sequences=True)(combined_train)
lstm_out_train = LSTM(64, return_sequences=False)(lstm_out_train)
dropout_train = Dropout(dropout_rate)(lstm_out_train)
dense_1_train = Dense(128, activation='relu')(dropout_train)
dropout_train_2 = Dropout(dropout_rate)(dense_1_train)
dense_2_train = Dense(72, activation='relu')(dropout_train_2)
output_train = Dense(map_height * map_width, activation='linear', name='output')(dense_2_train)
output_train = Reshape((map_height, map_width))(output_train)

# Training model
model_train = Model(inputs=list(inputs_train.values()), outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Concatenating all inputs for testing, including injectivity
merged_test = Concatenate(axis=-1)(list(cnn_test.values()))
flattened_merged_test = TimeDistributed(Flatten())(merged_test)
combined_test = Concatenate(axis=-1)([flattened_merged_test, repeated_coordinates_test, repeated_tvd_test, inputs_test['injectivity']])
lstm_out_test = LSTM(128, return_sequences=False)(combined_test)
dropout_test = Dropout(dropout_rate)(lstm_out_test)
dense_1_test = Dense(128, activation='relu')(dropout_test)
dropout_test_2 = Dropout(dropout_rate)(dense_1_test)
dense_2_test = Dense(72, activation='relu')(dropout_test_2)
output_test = Dense(map_height * map_width, activation='linear', name='output')(dense_2_test)
output_test = Reshape((map_height, map_width))(output_test)

# Testing model
model_test = Model(inputs=list(inputs_test.values()), outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Load and preprocess data (Assuming loaded_tensors and injectivity_array are already defined)
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Transforming df_X_Y and df_TVD to numpy arrays of required shape
coordinates_data = np.array(df_X_Y.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))
tvd_data = np.array(df_TVD.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))

# Ensuring data shapes match expected shape
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Проверка на наличие пропусков и заполнение нулями или средними значениями
oil_data = np.nan_to_num(oil_data)
water_data = np.nan_to_num(water_data)
reservoir_pressure_data = np.nan_to_num(reservoir_pressure_data)
bottomhole_pressure_data = np.nan_to_num(bottomhole_pressure_data)
injectivity_data = np.nan_to_num(injectivity_array.reshape((batch_size, total_time_steps, 4)))

# Масштабирование данных
scalers = {}
for data_name, data in zip(['oil', 'water', 'reservoir_pressure', 'bottomhole_pressure', 'injectivity'],
                           [oil_data, water_data, reservoir_pressure_data, bottomhole_pressure_data, injectivity_data]):
    scalers[data_name] = MinMaxScaler()
    data_reshaped = data.reshape(-1, data.shape[-1])
    scaled_data = scalers[data_name].fit_transform(data_reshaped)
    data[:] = scaled_data.reshape(data.shape)

# Splitting data into training and test sets
oil_train, oil_test = oil_data[:, :train_time_steps], oil_data[:, train_time_steps:]
water_train, water_test = water_data[:, :train_time_steps], water_data[:, train_time_steps:]
reservoir_pressure_train, reservoir_pressure_test = reservoir_pressure_data[:, :train_time_steps], reservoir_pressure_data[:, train_time_steps:]
bottomhole_pressure_train, bottomhole_pressure_test = bottomhole_pressure_data[:, :train_time_steps], bottomhole_pressure_data[:, train_time_steps:]

# Splitting injectivity data into training and test sets
injectivity_train, injectivity_test = injectivity_data[:, :train_time_steps], injectivity_data[:, train_time_steps:]

# Define output for training and testing
output_train_data = oil_data[:, :train_time_steps, :, :, 0].reshape((batch_size, train_time_steps, map_height, map_width))
output_test_data = oil_data[:, train_time_steps:, :, :, 0].reshape((batch_size, test_time_steps, map_height, map_width))

# Adding EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model with validation
history = model_train.fit(
    [oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train, coordinates_data, tvd_data, injectivity_train],
    output_train_data,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# Parameters
total_time_steps = 121
train_time_steps = 84
test_time_steps = total_time_steps - train_time_steps
map_height = 6
map_width = 6
batch_size = 1
dropout_rate = 0.3

# Function to create CNN layers for input tensors
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_tensor)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Input tensors for training model
inputs_train = {
    'oil': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(train_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Input tensors for testing model
inputs_test = {
    'oil': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(test_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Creating CNN layers for training and testing
cnn_train = {name: create_cnn(tensor) for name, tensor in inputs_train.items() if name not in ['coordinates', 'tvd', 'injectivity']}
cnn_test = {name: create_cnn(tensor) for name, tensor in inputs_test.items() if name not in ['coordinates', 'tvd', 'injectivity']}

# Flatten and repeat coordinates and TVD for training
flattened_coordinates_train = Flatten()(inputs_train['coordinates'])
repeated_coordinates_train = RepeatVector(train_time_steps)(flattened_coordinates_train)
flattened_tvd_train = Flatten()(inputs_train['tvd'])
repeated_tvd_train = RepeatVector(train_time_steps)(flattened_tvd_train)

# Flatten and repeat coordinates and TVD for testing
flattened_coordinates_test = Flatten()(inputs_test['coordinates'])
repeated_coordinates_test = RepeatVector(test_time_steps)(flattened_coordinates_test)
flattened_tvd_test = Flatten()(inputs_test['tvd'])
repeated_tvd_test = RepeatVector(test_time_steps)(flattened_tvd_test)

# Concatenating all inputs for training, including injectivity
merged_train = Concatenate(axis=-1)(list(cnn_train.values()))
flattened_merged_train = TimeDistributed(Flatten())(merged_train)
combined_train = Concatenate(axis=-1)([flattened_merged_train, repeated_coordinates_train, repeated_tvd_train, inputs_train['injectivity']])
# Example of adding more layers to the LSTM model
lstm_out_train = LSTM(128, return_sequences=True)(combined_train)
lstm_out_train = LSTM(64, return_sequences=False)(lstm_out_train)
dropout_train = Dropout(dropout_rate)(lstm_out_train)
dense_1_train = Dense(128, activation='relu')(dropout_train)
dropout_train_2 = Dropout(dropout_rate)(dense_1_train)
dense_2_train = Dense(72, activation='relu')(dropout_train_2)
output_train = Dense(map_height * map_width, activation='linear', name='output')(dense_2_train)
output_train = Reshape((map_height, map_width))(output_train)

# Training model
model_train = Model(inputs=list(inputs_train.values()), outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Concatenating all inputs for testing, including injectivity
merged_test = Concatenate(axis=-1)(list(cnn_test.values()))
flattened_merged_test = TimeDistributed(Flatten())(merged_test)
combined_test = Concatenate(axis=-1)([flattened_merged_test, repeated_coordinates_test, repeated_tvd_test, inputs_test['injectivity']])
lstm_out_test = LSTM(128, return_sequences=False)(combined_test)
dropout_test = Dropout(dropout_rate)(lstm_out_test)
dense_1_test = Dense(128, activation='relu')(dropout_test)
dropout_test_2 = Dropout(dropout_rate)(dense_1_test)
dense_2_test = Dense(72, activation='relu')(dropout_test_2)
output_test = Dense(map_height * map_width, activation='linear', name='output')(dense_2_test)
output_test = Reshape((map_height, map_width))(output_test)

# Testing model
model_test = Model(inputs=list(inputs_test.values()), outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Load and preprocess data (Assuming loaded_tensors and injectivity_array are already defined)
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Transforming df_X_Y and df_TVD to numpy arrays of required shape
coordinates_data = np.array(df_X_Y.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))
tvd_data = np.array(df_TVD.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))

# Ensuring data shapes match expected shape
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Проверка на наличие пропусков и заполнение нулями или средними значениями
oil_data = np.nan_to_num(oil_data)
water_data = np.nan_to_num(water_data)
reservoir_pressure_data = np.nan_to_num(reservoir_pressure_data)
bottomhole_pressure_data = np.nan_to_num(bottomhole_pressure_data)
injectivity_data = np.nan_to_num(injectivity_array.reshape((batch_size, total_time_steps, 4)))

# Масштабирование данных
scalers = {}
for data_name, data in zip(['oil', 'water', 'reservoir_pressure', 'bottomhole_pressure', 'injectivity'],
                           [oil_data, water_data, reservoir_pressure_data, bottomhole_pressure_data, injectivity_data]):
    scalers[data_name] = MinMaxScaler()
    data_reshaped = data.reshape(-1, data.shape[-1])
    scaled_data = scalers[data_name].fit_transform(data_reshaped)
    data[:] = scaled_data.reshape(data.shape)

# Splitting data into training and test sets
oil_train, oil_test = oil_data[:, :train_time_steps], oil_data[:, train_time_steps:]
water_train, water_test = water_data[:, :train_time_steps], water_data[:, train_time_steps:]
reservoir_pressure_train, reservoir_pressure_test = reservoir_pressure_data[:, :train_time_steps], reservoir_pressure_data[:, train_time_steps:]
bottomhole_pressure_train, bottomhole_pressure_test = bottomhole_pressure_data[:, :train_time_steps], bottomhole_pressure_data[:, train_time_steps:]

# Splitting injectivity data into training and test sets
injectivity_train, injectivity_test = injectivity_data[:, :train_time_steps], injectivity_data[:, train_time_steps:]

# Define output for training and testing
output_train_data = oil_data[:, :train_time_steps, :, :, 0].reshape((batch_size, train_time_steps, map_height, map_width))
output_test_data = oil_data[:, train_time_steps:, :, :, 0].reshape((batch_size, test_time_steps, map_height, map_width))

# Adding EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model with validation
history = model_train.fit(
    [oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train, coordinates_data, tvd_data, injectivity_train],
    output_train_data,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# Parameters
total_time_steps = 121
train_time_steps = 84
test_time_steps = total_time_steps - train_time_steps
map_height = 6
map_width = 6
batch_size = 1
dropout_rate = 0.3

# Function to create CNN layers for input tensors
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_tensor)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Input tensors for training model
inputs_train = {
    'oil': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(train_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Input tensors for testing model
inputs_test = {
    'oil': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(test_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Creating CNN layers for training and testing
cnn_train = {name: create_cnn(tensor) for name, tensor in inputs_train.items() if name not in ['coordinates', 'tvd', 'injectivity']}
cnn_test = {name: create_cnn(tensor) for name, tensor in inputs_test.items() if name not in ['coordinates', 'tvd', 'injectivity']}

# Flatten and repeat coordinates and TVD for training
flattened_coordinates_train = Flatten()(inputs_train['coordinates'])
repeated_coordinates_train = RepeatVector(train_time_steps)(flattened_coordinates_train)
flattened_tvd_train = Flatten()(inputs_train['tvd'])
repeated_tvd_train = RepeatVector(train_time_steps)(flattened_tvd_train)

# Flatten and repeat coordinates and TVD for testing
flattened_coordinates_test = Flatten()(inputs_test['coordinates'])
repeated_coordinates_test = RepeatVector(test_time_steps)(flattened_coordinates_test)
flattened_tvd_test = Flatten()(inputs_test['tvd'])
repeated_tvd_test = RepeatVector(test_time_steps)(flattened_tvd_test)

# Concatenating all inputs for training, including injectivity
merged_train = Concatenate(axis=-1)(list(cnn_train.values()))
flattened_merged_train = TimeDistributed(Flatten())(merged_train)
combined_train = Concatenate(axis=-1)([flattened_merged_train, repeated_coordinates_train, repeated_tvd_train, inputs_train['injectivity']])
# Example of adding more layers to the LSTM model
lstm_out_train = LSTM(128, return_sequences=True)(combined_train)
lstm_out_train = LSTM(64, return_sequences=False)(lstm_out_train)
dropout_train = Dropout(dropout_rate)(lstm_out_train)
dense_1_train = Dense(128, activation='relu')(dropout_train)
dropout_train_2 = Dropout(dropout_rate)(dense_1_train)
dense_2_train = Dense(72, activation='relu')(dropout_train_2)
output_train = Dense(map_height * map_width, activation='linear', name='output')(dense_2_train)
output_train = Reshape((map_height, map_width))(output_train)

# Training model
model_train = Model(inputs=list(inputs_train.values()), outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Concatenating all inputs for testing, including injectivity
merged_test = Concatenate(axis=-1)(list(cnn_test.values()))
flattened_merged_test = TimeDistributed(Flatten())(merged_test)
combined_test = Concatenate(axis=-1)([flattened_merged_test, repeated_coordinates_test, repeated_tvd_test, inputs_test['injectivity']])
lstm_out_test = LSTM(128, return_sequences=False)(combined_test)
dropout_test = Dropout(dropout_rate)(lstm_out_test)
dense_1_test = Dense(128, activation='relu')(dropout_test)
dropout_test_2 = Dropout(dropout_rate)(dense_1_test)
dense_2_test = Dense(72, activation='relu')(dropout_test_2)
output_test = Dense(map_height * map_width, activation='linear', name='output')(dense_2_test)
output_test = Reshape((map_height, map_width))(output_test)

# Testing model
model_test = Model(inputs=list(inputs_test.values()), outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Load and preprocess data (Assuming loaded_tensors and injectivity_array are already defined)
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Transforming df_X_Y and df_TVD to numpy arrays of required shape
coordinates_data = np.array(df_X_Y.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))
tvd_data = np.array(df_TVD.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))

# Ensuring data shapes match expected shape
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Проверка на наличие пропусков и заполнение нулями или средними значениями
oil_data = np.nan_to_num(oil_data)
water_data = np.nan_to_num(water_data)
reservoir_pressure_data = np.nan_to_num(reservoir_pressure_data)
bottomhole_pressure_data = np.nan_to_num(bottomhole_pressure_data)
injectivity_data = np.nan_to_num(injectivity_array.reshape((batch_size, total_time_steps, 4)))

# Масштабирование данных
scalers = {}
for data_name, data in zip(['oil', 'water', 'reservoir_pressure', 'bottomhole_pressure', 'injectivity'],
                           [oil_data, water_data, reservoir_pressure_data, bottomhole_pressure_data, injectivity_data]):
    scalers[data_name] = MinMaxScaler()
    data_reshaped = data.reshape(-1, data.shape[-1])
    scaled_data = scalers[data_name].fit_transform(data_reshaped)
    data[:] = scaled_data.reshape(data.shape)

# Splitting data into training and test sets
oil_train, oil_test = oil_data[:, :train_time_steps], oil_data[:, train_time_steps:]
water_train, water_test = water_data[:, :train_time_steps], water_data[:, train_time_steps:]
reservoir_pressure_train, reservoir_pressure_test = reservoir_pressure_data[:, :train_time_steps], reservoir_pressure_data[:, train_time_steps:]
bottomhole_pressure_train, bottomhole_pressure_test = bottomhole_pressure_data[:, :train_time_steps], bottomhole_pressure_data[:, train_time_steps:]

# Splitting injectivity data into training and test sets
injectivity_train, injectivity_test = injectivity_data[:, :train_time_steps], injectivity_data[:, train_time_steps:]

# Define output for training and testing
output_train_data = oil_data[:, :train_time_steps, :, :, 0].reshape((batch_size, train_time_steps, map_height, map_width))
output_test_data = oil_data[:, train_time_steps:, :, :, 0].reshape((batch_size, test_time_steps, map_height, map_width))

# Adding EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model with validation
history = model_train.fit(
    [oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train, coordinates_data, tvd_data, injectivity_train],
    output_train_data,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Concatenate, RepeatVector, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# Parameters
total_time_steps = 121
train_time_steps = 84
test_time_steps = total_time_steps - train_time_steps
map_height = 6
map_width = 6
batch_size = 1
dropout_rate = 0.3

# Function to create CNN layers for input tensors
def create_cnn(input_tensor):
    x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(input_tensor)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)
    x = TimeDistributed(MaxPooling2D((2, 2)))(x)
    x = TimeDistributed(Flatten())(x)
    return x

# Input tensors for training model
inputs_train = {
    'oil': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(train_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(train_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Input tensors for testing model
inputs_test = {
    'oil': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_oil'),
    'water': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_water'),
    'reservoir_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_reservoir_pressure'),
    'bottomhole_pressure': Input(shape=(test_time_steps, map_height, map_width, 1), name='input_bottomhole_pressure'),
    'coordinates': Input(shape=(map_height, map_width, 2), name='input_coordinates'),
    'tvd': Input(shape=(map_height, map_width, 2), name='input_tvd'),
    'injectivity': Input(shape=(test_time_steps, 4), name='input_injectivity')  # добавляем приемистость
}

# Creating CNN layers for training and testing
cnn_train = {name: create_cnn(tensor) for name, tensor in inputs_train.items() if name not in ['coordinates', 'tvd', 'injectivity']}
cnn_test = {name: create_cnn(tensor) for name, tensor in inputs_test.items() if name not in ['coordinates', 'tvd', 'injectivity']}

# Flatten and repeat coordinates and TVD for training
flattened_coordinates_train = Flatten()(inputs_train['coordinates'])
repeated_coordinates_train = RepeatVector(train_time_steps)(flattened_coordinates_train)
flattened_tvd_train = Flatten()(inputs_train['tvd'])
repeated_tvd_train = RepeatVector(train_time_steps)(flattened_tvd_train)

# Flatten and repeat coordinates and TVD for testing
flattened_coordinates_test = Flatten()(inputs_test['coordinates'])
repeated_coordinates_test = RepeatVector(test_time_steps)(flattened_coordinates_test)
flattened_tvd_test = Flatten()(inputs_test['tvd'])
repeated_tvd_test = RepeatVector(test_time_steps)(flattened_tvd_test)

# Concatenating all inputs for training, including injectivity
merged_train = Concatenate(axis=-1)(list(cnn_train.values()))
flattened_merged_train = TimeDistributed(Flatten())(merged_train)
combined_train = Concatenate(axis=-1)([flattened_merged_train, repeated_coordinates_train, repeated_tvd_train, inputs_train['injectivity']])
# Example of adding more layers to the LSTM model
lstm_out_train = LSTM(128, return_sequences=True)(combined_train)
lstm_out_train = LSTM(64, return_sequences=False)(lstm_out_train)
dropout_train = Dropout(dropout_rate)(lstm_out_train)
dense_1_train = Dense(128, activation='relu')(dropout_train)
dropout_train_2 = Dropout(dropout_rate)(dense_1_train)
dense_2_train = Dense(72, activation='relu')(dropout_train_2)
output_train = Dense(map_height * map_width, activation='linear', name='output')(dense_2_train)
output_train = Reshape((map_height, map_width))(output_train)

# Training model
model_train = Model(inputs=list(inputs_train.values()), outputs=output_train)
model_train.compile(optimizer='adam', loss='mse')

# Concatenating all inputs for testing, including injectivity
merged_test = Concatenate(axis=-1)(list(cnn_test.values()))
flattened_merged_test = TimeDistributed(Flatten())(merged_test)
combined_test = Concatenate(axis=-1)([flattened_merged_test, repeated_coordinates_test, repeated_tvd_test, inputs_test['injectivity']])
lstm_out_test = LSTM(128, return_sequences=False)(combined_test)
dropout_test = Dropout(dropout_rate)(lstm_out_test)
dense_1_test = Dense(128, activation='relu')(dropout_test)
dropout_test_2 = Dropout(dropout_rate)(dense_1_test)
dense_2_test = Dense(72, activation='relu')(dropout_test_2)
output_test = Dense(map_height * map_width, activation='linear', name='output')(dense_2_test)
output_test = Reshape((map_height, map_width))(output_test)

# Testing model
model_test = Model(inputs=list(inputs_test.values()), outputs=output_test)
model_test.compile(optimizer='adam', loss='mse')

# Load and preprocess data (Assuming loaded_tensors and injectivity_array are already defined)
oil_data = np.expand_dims(loaded_tensors['debit_neft_series.tfrecord'].numpy(), axis=-1)
water_data = np.expand_dims(loaded_tensors['debit_water_series.tfrecord'].numpy(), axis=-1)
reservoir_pressure_data = np.expand_dims(loaded_tensors['debit_plast_pressure_series.tfrecord'].numpy(), axis=-1)
bottomhole_pressure_data = np.expand_dims(loaded_tensors['debit_zaboy_pressure_series.tfrecord'].numpy(), axis=-1)

# Transforming df_X_Y and df_TVD to numpy arrays of required shape
coordinates_data = np.array(df_X_Y.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))
tvd_data = np.array(df_TVD.applymap(lambda x: list(x)).values.tolist()).reshape((batch_size, map_height, map_width, 2))

# Ensuring data shapes match expected shape
oil_data = oil_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
water_data = water_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
reservoir_pressure_data = reservoir_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))
bottomhole_pressure_data = bottomhole_pressure_data.reshape((batch_size, total_time_steps, map_height, map_width, 1))

# Проверка на наличие пропусков и заполнение нулями или средними значениями
oil_data = np.nan_to_num(oil_data)
water_data = np.nan_to_num(water_data)
reservoir_pressure_data = np.nan_to_num(reservoir_pressure_data)
bottomhole_pressure_data = np.nan_to_num(bottomhole_pressure_data)
injectivity_data = np.nan_to_num(injectivity_array.reshape((batch_size, total_time_steps, 4)))

# Масштабирование данных
scalers = {}
for data_name, data in zip(['oil', 'water', 'reservoir_pressure', 'bottomhole_pressure', 'injectivity'],
                           [oil_data, water_data, reservoir_pressure_data, bottomhole_pressure_data, injectivity_data]):
    scalers[data_name] = MinMaxScaler()
    data_reshaped = data.reshape(-1, data.shape[-1])
    scaled_data = scalers[data_name].fit_transform(data_reshaped)
    data[:] = scaled_data.reshape(data.shape)

# Splitting data into training and test sets
oil_train, oil_test = oil_data[:, :train_time_steps], oil_data[:, train_time_steps:]
water_train, water_test = water_data[:, :train_time_steps], water_data[:, train_time_steps:]
reservoir_pressure_train, reservoir_pressure_test = reservoir_pressure_data[:, :train_time_steps], reservoir_pressure_data[:, train_time_steps:]
bottomhole_pressure_train, bottomhole_pressure_test = bottomhole_pressure_data[:, :train_time_steps], bottomhole_pressure_data[:, train_time_steps:]

# Splitting injectivity data into training and test sets
injectivity_train, injectivity_test = injectivity_data[:, :train_time_steps], injectivity_data[:, train_time_steps:]

# Define output for training and testing
output_train_data = oil_data[:, :train_time_steps, :, :, 0].reshape((batch_size, train_time_steps, map_height, map_width))
output_test_data = oil_data[:, train_time_steps:, :, :, 0].reshape((batch_size, test_time_steps, map_height, map_width))

# Adding EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model with validation
history = model_train.fit(
    [oil_train, water_train, reservoir_pressure_train, bottomhole_pressure_train, coordinates_data, tvd_data, injectivity_train],
    output_train_data,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_performance(val_target, val_output):
  val_target = val_target.numpy()
  val_output = val_output.numpy()

  output_shape = val_target.shape[1:]

  fig, axes = plt.subplots(output_shape[0], output_shape[1], figsize=(20, 20))
  fig.suptitle('Model Performance on Validation Time Steps', fontsize=16)

  time_steps = np.arange(val_output.shape[0])

  for i in range(output_shape[0]):
    for j in range(output_shape[1]):
      ax = axes[i, j]
      real_values = val_target[:, i, j]
      predicted_values = val_output[:, i, j]

      ax.plot(time_steps, real_values, label='Real', color='blue')
      ax.plot(time_steps, predicted_values, label='Predicted', color='red')
      ax.set_title(f'Value at ({i}, {j})')
      ax.set_ylim(-10, 160)
      ax.legend()

  plt.tight_layout(rect=[0, 0, 1, 0.96])
  plt.show()

plot_model_performance(val_target, val_output)